In [ ]:
# Map Page

> The page of the app that generates the map view of the data.  

In [ ]:
#| default_exp pages/map_page

In [ ]:
#| export
# TODO minimise these imports a bit where possible
from nbdev.showdoc import *
# TODO work out how to get around below hack
try:
    from SolomonIslandsDataMap.load_data import *
    from SolomonIslandsDataMap.dash_components import *
except: 
    from load_data import *
    from dash_components import *
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # Unless this is used graphs will not be dynamic?
import numpy as np
from fastcore.test import *
from dash import Dash, dcc, Output, Input, State, html, Patch, ctx, register_page  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
from dash_bootstrap_templates import load_figure_template
import dash_mantine_components as dmc
import os

In [ ]:
#| export
register_page(__name__, 
                    path='/',
                    title='Data Map',
                    name='Data Map')

In [ ]:
# TODO load the data and dropdown

Setup some of  the key data contiaining objects

In [ ]:
#| export
mytitle = dcc.Markdown(children="## " + list(cen_vars.keys())[0] + " by " + geos[0]) # TODO This needs a default title
map_graph = dcc.Graph(figure=define_map(sol_geo), selectedData=None,)
# TODO entire accordian will need to be the child
selectedBarGraph = dcc.Graph(figure = gen_bar_plot(sol_geo, sol_geo.geo_levels[0], 
                                               "Key Statistics", 'Total Households'),
                            id = 'bar_graph')
# Selections tracks the currently selected map locations
#selection = dcc.Store(id = 'selection',data = {})

In [ ]:
sol_geo.locations[sol_geo.geo_levels[0]]

Index(['Central', 'Choiseul', 'Guadalcanal', 'Honiara', 'Isabel',
       'Makira-Ulawa', 'Malaita', 'Rennell-Bell', 'Temotu', 'Western'],
      dtype='object', name='geo_name')

## Customize the Layout

In [ ]:
#| export
def layout():
    return dbc.Container([mytitle,
                        map_graph,
                        selectedBarGraph,
                        ])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()